# prepare lc questions

## load data

In [ ]:
# create a mini dataset

# import itertools
# from pathlib import Path

# in_path  = Path("/content/LeetCodeDataset-v0.1.0-train.jsonl")
# out_path = Path("/content/LeetCodeDataset-sample20.jsonl")

# # Copy the first 20 raw lines -------------------------------------------------
# with in_path.open("r", encoding="utf-8") as fin, \
#      out_path.open("w", encoding="utf-8") as fout:

#     for line in itertools.islice(fin, 20):
#         fout.write(line)

# print(f"Saved the first 20 problems to {out_path}")

In [2]:
import json
from datasets import Dataset

/Users/jg/Documents/academic/grad/2025S/1508/codegen/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
with open("LeetCodeDataset-sample20.jsonl", encoding="utf-8") as f:
    rows = [json.loads(line) for line in f]

ds = Dataset.from_list(rows)
print(ds)

Dataset({
    features: ['task_id', 'prompt', 'completion', 'entry_point', 'test', 'query', 'response', 'input_output', 'meta'],
    num_rows: 20
})


In [5]:
q0 = ds[0]

In [6]:
print(q0)

{'task_id': 'two-sum', 'prompt': "from typing import *\n\nfrom functools import *\nfrom collections import *\nfrom itertools import *\nfrom heapq import *\nfrom bisect import *\nfrom string import *\nfrom operator import *\nfrom math import *\nimport math\nimport datetime\ninf = float('inf')\n", 'completion': 'class Solution:\n    def twoSum(self, nums: List[int], target: int) -> List[int]:\n        m = {}\n        for i, x in enumerate(nums):\n            y = target - x\n            if y in m:\n                return [m[y], i]\n            m[x] = i\n', 'entry_point': 'Solution().twoSum', 'test': 'def check(candidate):\n    assert candidate(nums = [2,7,11,15], target = 9) == [0,1]\n    assert candidate(nums = [3,2,4], target = 6) == [1,2]\n    assert candidate(nums = [3,3], target = 6) == [0,1]\n', 'query': 'Given an array of integers nums\xa0and an integer target, return indices of the two numbers such that they add up to target.\nYou may assume that each input would have exactly one 

## prompt template

In [7]:
PROMPT_TEMPLATE_EXAMPLE = """
You are a senior Python instructor. For the given LeetCode problem, do the following:
1. First output a clear, numbered step-by-step plan.
2. Then write the **final Python function** ONLY between
   the tags:  ### BEGIN SOLUTION  ...  ### END SOLUTION
3. After the END tag, give a one-line time/space complexity note.

Problem:
“Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.
You may assume that each input would have exactly one solution, and you may not use the same element twice.
You can return the answer in any order.
Example: nums = [2,7,11,15], target = 9 → [0,1]”
"""

In [8]:
PROMPT_TEMPLATE = """
You are a senior Python instructor. For the given LeetCode problem, do the following:
Write the **final Python function** ONLY between the tags:  ### BEGIN SOLUTION  ...  ### END SOLUTION

Problem:
{problem}

Example:
{examples}
"""

In [9]:
q0_problem = q0["query"].split("Please complete")[0].strip()
q0_examples = "\n\n".join(
    f"Input:  {item['input']}\nOutput: {item['output']}"
    for item in q0["input_output"][:2]
)
q0_prompt = PROMPT_TEMPLATE.format(problem=q0_problem, examples=q0_examples)
print(q0_prompt)


You are a senior Python instructor. For the given LeetCode problem, do the following:
Write the **final Python function** ONLY between the tags:  ### BEGIN SOLUTION  ...  ### END SOLUTION

Problem:
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.
You may assume that each input would have exactly one solution, and you may not use the same element twice.
You can return the answer in any order.

Example:
Input:  nums = [2,7,11,15], target = 9
Output: [0,1]

Input:  nums = [3,2,4], target = 6
Output: [1,2]



## test template

In [10]:
print(q0['entry_point'])

Solution().twoSum


In [11]:
print(q0['test'])

def check(candidate):
    assert candidate(nums = [2,7,11,15], target = 9) == [0,1]
    assert candidate(nums = [3,2,4], target = 6) == [1,2]
    assert candidate(nums = [3,3], target = 6) == [0,1]



In [12]:
print(q0['input_output'])

[{'input': 'nums = [2,7,11,15], target = 9', 'output': '[0,1]'}, {'input': 'nums = [3,2,4], target = 6', 'output': '[1,2]'}, {'input': 'nums = [3,3], target = 6', 'output': '[0,1]'}]


In [13]:
TEST_SNIPPET_EXAMPLE = """
import pytest
from solution import two_sum

def test_example():
    assert two_sum([2,7,11,15], 9) == [0,1]
"""

In [14]:
TEST_SNIPPET_TEMPLATE = """
import pytest
from solution import two_sum as candidate

{test_code}
"""

In [15]:
print(q0['test'])

def check(candidate):
    assert candidate(nums = [2,7,11,15], target = 9) == [0,1]
    assert candidate(nums = [3,2,4], target = 6) == [1,2]
    assert candidate(nums = [3,3], target = 6) == [0,1]



In [16]:
q0_test_snippet = TEST_SNIPPET_TEMPLATE.format(test_code=q0['test'])
print(q0_test_snippet)


import pytest
from solution import two_sum as candidate

def check(candidate):
    assert candidate(nums = [2,7,11,15], target = 9) == [0,1]
    assert candidate(nums = [3,2,4], target = 6) == [1,2]
    assert candidate(nums = [3,3], target = 6) == [0,1]




note: needs name the generated code: sol
e.g. `def sol` instead of `def two_sum`

In [17]:
import textwrap

def convert_check_to_pytest(src: str,
                            import_line: str = "from solution import sol as candidate",
                            test_name: str = "test") -> str:
    """
    Turn a LeetCode 'check(candidate)' block into a pytest-friendly test_…()
    with no fixture parameters.
    """
    lines = src.splitlines()

    # find the line that starts 'def check('
    for i, line in enumerate(lines):
        if line.lstrip().startswith("def check"):
            header_idx = i
            break
    else:
        raise ValueError("No 'def check' block found.")

    # collects assert
    body_lines = lines[header_idx + 1:]

    # adjust indentation
    dedented_body = textwrap.dedent("\n".join(body_lines)).rstrip()

    # bulid test script
    new_test = f"""{import_line}

def {test_name}():
{textwrap.indent(dedented_body, '    ')}
"""

    return new_test.strip() + "\n"

fixed_test_code = convert_check_to_pytest(q0["test"])

print(fixed_test_code)

from solution import sol as candidate

def test():
    assert candidate(nums = [2,7,11,15], target = 9) == [0,1]
    assert candidate(nums = [3,2,4], target = 6) == [1,2]
    assert candidate(nums = [3,3], target = 6) == [0,1]



## run test

In [18]:
import tempfile
import pathlib
import subprocess

def run_pytest(code: str, test_code: str):
    tmp = pathlib.Path(tempfile.mkdtemp())
    (tmp/"solution.py").write_text(code)
    (tmp/"test_solution.py").write_text(test_code)
    proc = subprocess.run(
        ["pytest", "-q"], cwd=tmp, capture_output=True, text=True
    )
    return proc.returncode == 0, proc.stdout + proc.stderr

In [19]:
code = """
def sol(nums, target):
    for i in range(len(nums)):
        for j in range(i + 1, len(nums)):
            if nums[i] + nums[j] == target:
                return [i, j]
"""

In [21]:
passed, log = run_pytest(code, fixed_test_code)
print(passed, log)

True .                                                                        [100%]
1 passed in 0.01s

